In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import matplotlib.pyplot as plt
from IPython.display import Image, display
import random
import math

import tensorflow as tf
from keras.preprocessing.image import load_img,img_to_array,ImageDataGenerator
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.utils import plot_model 
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten, Activation,Concatenate
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.utils import to_categorical 
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras import backend, models
from sklearn.metrics import confusion_matrix

from tensorflow.keras.applications import VGG16, MobileNet
from keras.applications.vgg16 import preprocess_input

****Let's create Tenosrflow datasets****

In [ ]:
image_size = (224, 224)

train_path = '../input/100-bird-species/birds/train'
valid_path = '../input/100-bird-species/birds/valid'
test_path = '../input/100-bird-species/birds/test'

We will use diffirent Image Generators because we want to apply Data Augmentation techniques only to training dataset. This techniques are used in order to decrease overfiting of neural networks

In [ ]:
default_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
augmentation_datagen = ImageDataGenerator(rescale=1./255, 
                                          rotation_range=20,
                                          width_shift_range=0.2,
                                          height_shift_range=0.2,
                                          zoom_range=0.2,
                                          horizontal_flip=True,
                                          fill_mode='nearest') 

In [ ]:
train_data = augmentation_datagen.flow_from_directory(train_path,
                                                      target_size=image_size)

In [ ]:
validation_data = default_datagen.flow_from_directory(valid_path,
                                                      target_size=image_size)

In [ ]:
test_data = default_datagen.flow_from_directory(test_path, 
                                                target_size=image_size)

Let's have a look at and some random bird's species and their bird species bird species photos

In [ ]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(train_path)
class_names = train_dataset.class_names

plt.figure(figsize=(12, 12))
for images, labels in train_dataset.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

****Now we will use a base model with weights pretrained on Imagenet****

In [ ]:
base_mobilenet = MobileNet(weights = 'imagenet',
                           include_top = False,
                           input_shape = (224,224,3))

Let's freeze the weights of our base model

In [ ]:
base_mobilenet.trainable = False 

In [ ]:
model = Sequential()
# We add the base model as a new layer
model.add(base_mobilenet)
model.add(Flatten()) 
model.add(Activation('relu'))
# 275 - number of classes
model.add(Dense(275)) 
model.add(Activation('softmax'))

Now let's check the updated model summary. You can see the new layers from below. This layers add 13.8 million of new parametrs to train.

In [ ]:
model.summary()

In [ ]:
# we want our learning rate to be adaptive
opt = tf.keras.optimizers.SGD(lr=0.001,
                              momentum=0.6,
                              nesterov=True)

model.compile(optimizer = opt,
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

In [ ]:
history = model.fit(
    train_data, 
    steps_per_epoch = 600, 
    epochs = 12,
    validation_data = validation_data,
    validation_steps = 32,
    verbose = 1,
    callbacks=[EarlyStopping(monitor='val_accuracy',
                             patience=3,
                             restore_best_weights=True),
               ReduceLROnPlateau(monitor='val_loss',
                                 factor=0.7,
                                 patience=2,
                                 verbose=1)]
) 

The time of fiting was long enough but the validation accuracy was worth it

In [ ]:
model.evaluate(test_data)

Accuracy **94.4%** on test data is a high result. Given the fact that this neural network is the first in my life, I guess I've got the right be glad. If someone has feedback or some pieces of advice I'll be glad to hear from you